In [2]:
!pip install --upgrade pip wheel setuptools

# Go 없이 설치 가능한 예전 버전 사용
!pip install "wandb<0.23"
!pip install numpy
!pip install pandas
!pip install torch
!pip install scikit-learn
!pip install tqdm
!pip install google
!pip install google-genai pandas --upgrade
!pip install s3fs
!pip install numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.3 MB/s  0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 134.5 MB/s  0:00:00
Using cached gitpython-3.1.45-py3-none-any.whl (208 kB)
Using cached gitdb-4.0.12-py3-none-any.whl (62 kB)
Using cached smmap-5.0.2-py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [wandb]32m4/5 [wandb]-sdk]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 36.2 MB/s  0:00:10m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 69.2 MB/s  0:00:04m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

# VQVAE

## Environment Setting

In [2]:
import os
import wandb

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split

from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


cuda


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
csv_filename = 'spy_2023_2024.csv'
csv_filepath = '/content/drive/MyDrive/2025 ML Project/datasets/spy_data.csv'

save_dir = 'prepared_data/'

In [5]:
#wandb.finish()

In [6]:
#wandb.login()


## Data Preprocessing

In [7]:
# csv to DF
data = pd.read_csv(csv_filename)
data["date"] = pd.to_datetime(data["date"])

In [8]:
data

,Unnamed: 0,date,1. open,2. high,3. low,4. close,5. volume
0,0,2023-01-03 09:30:00,370.5398,372.5257,367.9274,367.9755,7391068.0
1,1,2023-01-03 10:00:00,368.0334,368.8142,366.4669,367.2332,4314212.0
2,2,2023-01-03 10:30:00,367.1850,368.3467,366.3946,368.2937,3567632.0
3,3,2023-01-03 11:00:00,368.3032,368.5732,365.8258,366.3639,4035712.0
4,4,2023-01-03 11:30:00,366.3656,366.3753,365.0256,365.4305,3691236.0
...,...,...,...,...,...,...,...
6521,6521,2024-12-31 13:30:00,579.9381,581.4202,579.8440,580.2603,1760552.0
6522,6522,2024-12-31 14:00:00,580.2801,581.4696,579.5069,579.8638,3210970.0
6523,6523,2024-12-31 14:30:00,579.8440,581.2516,579.3483,581.2417,3129664.0
6524,6524,2024-12-31 15:00:00,581.2219,582.2033,580.0918,582.1340,3598139.0


In [9]:
## 하루를 기준으로 정규화
def normalize_per_day(group):
  ohlc = ['1. open', '2. high', '3. low', '4. close']

  min_val = group[ohlc].min().min()
  max_val = group[ohlc].max().max()

  if max_val - min_val > 0:
    group[ohlc] = (group[ohlc] - min_val) / (max_val - min_val)
  else:
    group[ohlc] = 0.5

  ## 일단 volume도 하루 단위로 정규화
  min_vol = group['5. volume'].min()
  max_vol = group['5. volume'].max()
  group['5. volume'] = (group['5. volume'] - min_vol) / (max_vol - min_vol)

  return group

normalized_data = data.groupby(data['date'].dt.date).apply(normalize_per_day)

In [10]:
normalized_data

Unnamed: 0                date   1. open   2. high    3. low  \
date                                                                            
2023-01-03 0              0 2023-01-03 09:30:00  0.760435  1.000000  0.445293   
           1              1 2023-01-03 10:00:00  0.458080  0.552270  0.269108   
           2              2 2023-01-03 10:30:00  0.355735  0.495874  0.260387   
           3              3 2023-01-03 11:00:00  0.490627  0.523198  0.191770   
           4              4 2023-01-03 11:30:00  0.256888  0.258058  0.095240   
...                     ...                 ...       ...       ...       ...   
2024-12-31 6521        6521 2024-12-31 13:30:00  0.095655  0.336026  0.080394   
           6522        6522 2024-12-31 14:00:00  0.151121  0.344037  0.025722   
           6523        6523 2024-12-31 14:30:00  0.080394  0.308682  0.000000   
           6524        6524 2024-12-31 15:00:00  0.303865  0.463031  0.120583   
           6525        6525 2024-12-31 15:30:00  0.451775  0.495175  0.088422   

                 4. close  5. volume  
date                                  
2023-01-03 0     0.451095   1.000000  
           1     0.361549   0.438819  
           2     0.489481   0.302651  
           3     0.256683   0.388024  
           4     0.144084   0.325195  
...                   ...        ...  
2024-12-31 6521  0.147910   0.012937  
           6522  0.083605   0.145332  
           6523  0.307076   0.137911  
           6524  0.451791   0.180674  
           6525  0.255632   1.000000  

[6526 rows x 7 columns]

In [11]:
normalized_data.head(14)

Unnamed: 0                date   1. open   2. high    3. low  \
date                                                                          
2023-01-03 0            0 2023-01-03 09:30:00  0.760435  1.000000  0.445293   
           1            1 2023-01-03 10:00:00  0.458080  0.552270  0.269108   
           2            2 2023-01-03 10:30:00  0.355735  0.495874  0.260387   
           3            3 2023-01-03 11:00:00  0.490627  0.523198  0.191770   
           4            4 2023-01-03 11:30:00  0.256888  0.258058  0.095240   
           5            5 2023-01-03 12:00:00  0.148741  0.220843  0.069666   
           6            6 2023-01-03 12:30:00  0.131876  0.254572  0.000000   
           7            7 2023-01-03 13:00:00  0.092924  0.215692  0.059195   
           8            8 2023-01-03 13:30:00  0.196415  0.251074  0.131297   
           9            9 2023-01-03 14:00:00  0.233630  0.278988  0.111525   
           10          10 2023-01-03 14:30:00  0.135953  0.339461  0.030122   
           11          11 2023-01-03 15:00:00  0.254572  0.385978  0.231302   
           12          12 2023-01-03 15:30:00  0.295274  0.403421  0.285961   
2023-01-04 13          13 2023-01-04 09:30:00  0.540805  0.709177  0.323125   

               4. close  5. volume  
date                                
2023-01-03 0   0.451095   1.000000  
           1   0.361549   0.438819  
           2   0.489481   0.302651  
           3   0.256683   0.388024  
           4   0.144084   0.325195  
           5   0.128969   0.096589  
           6   0.090595   0.131098  
           7   0.196415   0.000000  
           8   0.232472   0.008069  
           9   0.134783   0.086460  
           10  0.256876   0.366619  
           11  0.294103   0.294148  
           12  0.349920   0.981363  
2023-01-04 13  0.549326   0.480972

In [12]:
class SPYDataSet(Dataset):
  def __init__(self, data, features, chunk_size):
    self.chunk_size = chunk_size

    arr = data[features].to_numpy(dtype=np.float32)
    self.arr = arr
    self.N, self.C = arr.shape

    self.num_chunks = self.N // self.chunk_size

  def __len__(self):
    return self.num_chunks

  def __getitem__(self, idx: int):
    start = idx * self.chunk_size
    end = start + self.chunk_size

    x = self.arr[start:end]
    x = torch.from_numpy(x).float().T
    return {"x": x, "idx": idx}

In [13]:
feature_cols = ['1. open', '2. high', '3. low', '4. close', '5. volume']

ds = SPYDataSet(normalized_data, features=feature_cols, chunk_size=13)
print(len(ds))

502


In [14]:
ds[0] # C(5) * T(13)

{'x': tensor([[0.7604, 0.4581, 0.3557, 0.4906, 0.2569, 0.1487, 0.1319, 0.0929, 0.1964,
          0.2336, 0.1360, 0.2546, 0.2953],
         [1.0000, 0.5523, 0.4959, 0.5232, 0.2581, 0.2208, 0.2546, 0.2157, 0.2511,
          0.2790, 0.3395, 0.3860, 0.4034],
         [0.4453, 0.2691, 0.2604, 0.1918, 0.0952, 0.0697, 0.0000, 0.0592, 0.1313,
          0.1115, 0.0301, 0.2313, 0.2860],
         [0.4511, 0.3615, 0.4895, 0.2567, 0.1441, 0.1290, 0.0906, 0.1964, 0.2325,
          0.1348, 0.2569, 0.2941, 0.3499],
         [1.0000, 0.4388, 0.3027, 0.3880, 0.3252, 0.0966, 0.1311, 0.0000, 0.0081,
          0.0865, 0.3666, 0.2941, 0.9814]]),
 'idx': 0}

## VQ-VAE Modeling

In [15]:
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, latent_dim):
    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv1d(input_dim, hidden_dim, kernel_size=5, stride=2, padding=2),
        nn.ReLU(),
        nn.Conv1d(hidden_dim, hidden_dim, kernel_size=5, stride=1, padding=2),
        nn.ReLU(),
        nn.Conv1d(hidden_dim, latent_dim, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.AdaptiveAvgPool1d(1)
    )

  def forward(self, x):
    return self.conv(x)

In [16]:
class Decoder(nn.Module):
  def __init__(self, latent_dim, hidden_dim, output_dim):
    super().__init__()
    self.deconv = nn.Sequential(
          nn.ConvTranspose1d(latent_dim, hidden_dim, kernel_size=7, stride=1, padding=0),
          nn.ReLU(),
          nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.ConvTranspose1d(hidden_dim, hidden_dim, kernel_size=3, stride=2, padding=1, output_padding=0),
          nn.ReLU(),
          nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.Conv1d(hidden_dim, output_dim, kernel_size=3, stride=1, padding=1)
      )

  def forward(self, z_q):
      return self.deconv(z_q)

In [17]:
class VectorQuantizer(nn.Module):
  """ num_embeddings: K (codebook size)
      embedding_dim:  D (code dimension)
      commitment_cost: beta in the paper """

  def __init__(self, num_embeddings, embedding_dim, commitment_cost):
    super().__init__()
    self.num_embeddings = num_embeddings
    self.embedding = nn.Embedding(num_embeddings, embedding_dim)
    self.commitment_cost = commitment_cost
    self.embedding.weight.data.uniform_(-1/num_embeddings, 1/num_embeddings)

  def forward(self, z):
    B, D, T = z.shape
    z_perm = z.permute(0, 2, 1).contiguous()
    z_flattened = z_perm.view(-1, D)

    e = self.embedding.weight
    z_sq = (z_flattened ** 2).sum(dim=1, keepdim=True)
    e_sq = (e ** 2).sum(dim=1)
    ze = z_flattened @ e.t()
    distances = z_sq + e_sq.unsqueeze(0) - 2 * ze

    encoding_indices = torch.argmin(distances, dim=1)
    z_q = self.embedding(encoding_indices).view(B, T, D).permute(0, 2, 1).contiguous()

    codebook_loss =  F.mse_loss(z_q, z.detach())
    commitment_loss = self.commitment_cost * F.mse_loss(z_q.detach(), z)
    vq_loss = codebook_loss + 0.5 * commitment_loss

    z_q = z + (z_q - z).detach()

    indices_bt = encoding_indices.view(B, T)
    return z_q, vq_loss, indices_bt

In [18]:
class VQVAE(nn.Module):
  def __init__(self, input_dim, hidden_dim, latent_dim, num_embeddings, commitment_cost):
    super().__init__()
    self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
    self.vq = VectorQuantizer(num_embeddings, latent_dim, commitment_cost)
    self.decoder = Decoder(latent_dim, hidden_dim, input_dim)

  def forward(self, x):
    z_e = self.encoder(x)
    z_q, vq_loss, indices = self.vq(z_e)
    x_recon = self.decoder(z_q)
    return x_recon, vq_loss, indices, z_q

In [19]:
def evaluate(model, dataloader, device):
  index_list = []
  sum_recon, sum_vq = 0.0, 0.0
  n = 0

  model.eval()
  with torch.no_grad():
    for batch in tqdm(dataloader, desc="Evaluating", leave=False):
      X = batch["x"].to(device)

      x_recon, vq_loss, indices, _ = model(X)

      recon_loss = F.mse_loss(x_recon, X)

      batch_size = X.size(0)
      sum_recon += recon_loss.item() * batch_size
      sum_vq += vq_loss.item() * batch_size
      n += batch_size

      index_list.append(indices)

    mean_recon = sum_recon / max(n, 1)
    mean_vq = sum_vq / max(n, 1)
    sum_loss = mean_recon + mean_vq

    return mean_recon, mean_vq, sum_loss, index_list

In [20]:
import wandb
print(wandb.__file__)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/wandb/__init__.py


In [55]:
configs= [{
        "batch_size": 32,
        "learning_rate": 3e-4,
        "epochs": 200,
        "num_embeddings": 16,
        "commitment_cost":0.25
    }, {
        "batch_size": 32,
        "learning_rate": 3e-4,
        "epochs": 200,
        "num_embeddings": 16,
        "commitment_cost":0.4
    }, {
        "batch_size": 32,
        "learning_rate": 3e-4,
        "epochs": 200,
        "num_embeddings": 32,
        "commitment_cost":0.25
    }, {
        "batch_size": 32,
        "learning_rate": 3e-4,
        "epochs": 200,
        "num_embeddings": 32,
        "commitment_cost":0.4
    }, {
        "batch_size": 32,
        "learning_rate": 3e-4,
        "epochs": 200,
        "num_embeddings": 64,
        "commitment_cost":0.25
    }, {
        "batch_size": 32,
        "learning_rate": 3e-4,
        "epochs": 200,
        "num_embeddings": 64,
        "commitment_cost":0.4
    }]


In [56]:
for i in range(len(configs)):
    cfg = configs[i]
    wandb.init(
        project="2025 ML Project",
        mode="offline",
        entity="youani-korea-university",
        name="1 vector_8",
        config= cfg
    )
    
    N = len(ds)
    train_len = int(N * 0.7)
    val_len = int(N * 0.15)
    test_len = N - train_len - val_len
    
    ds_train, ds_val, ds_test = random_split(ds, [train_len, val_len, test_len])
    train_loader = DataLoader(ds_train, batch_size=wandb.config.batch_size, shuffle=True, drop_last=True)
    val_loader   = DataLoader(ds_val, batch_size=wandb.config.batch_size, shuffle=False)
    test_loader  = DataLoader(ds_test, batch_size=wandb.config.batch_size, shuffle=False)
    
    # parameter setting
    input_dim = 5
    hidden_dim = 64
    latent_dim = 8
    num_embeddings = wandb.config.num_embeddings
    commitment_cost = wandb.config.commitment_cost
    
    # model training
    lr = wandb.config.learning_rate
    epochs = wandb.config.epochs
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = VQVAE(input_dim, hidden_dim, latent_dim, num_embeddings, commitment_cost)
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    recon_hist, vq_hist = [], []
    
    best_val_loss = float("inf")
    best_model_path = 'prepared_data/best_model_epoch.pt'
    
    for epoch in range(1, epochs + 1):
      model.train()
      sum_recon, sum_vq = 0.0, 0.0
      n = 0
    
      for batch in tqdm(train_loader, desc="Train", leave=False):
        X = batch['x'].to(device)
        opt.zero_grad()
        x_recon, vq_loss, indices, _ = model(X)
    
        if x_recon.size(-1) != X.size(-1):
          print("Error: reconstruction size not equal to original")
          x_recon = F.interpolate(x_recon, size=X.size(-1), mode='linear', align_corners=False)
    
        recon_loss = F.mse_loss(x_recon, X)
        loss = recon_loss + vq_loss
    
        loss.backward()
        opt.step()
    
        batch_size = X.size(0)
        sum_recon += recon_loss.item() * batch_size
        sum_vq += vq_loss.item() * batch_size
        n += batch_size
    
      epoch_recon = sum_recon / max(n, 1)
      epoch_vq = sum_vq / max(n, 1)
    
      recon_hist.append(epoch_recon)
      vq_hist.append(epoch_vq)
    
      tqdm.write(f"[{epoch:03d}/{epochs:03d} Training] recon={epoch_recon:.6f} vq={epoch_vq:.6f}")
    
      val_recon, val_vq, val_loss, index_list = evaluate(model, val_loader, device)
      index_total = torch.cat(index_list).view(-1)
      usage_rate = len(index_total.unique()) / model.vq.num_embeddings
      wandb.log({
          f'valid_epoch_recon': val_recon,
          f'valid_epoch_vq': val_vq,
          f'usage_rate': usage_rate
      })
      tqdm.write(f"[{epoch:03d}/{epochs:03d} Validation] recon={val_recon:.6f} vq={val_vq:.6f}")
    
      if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_path = os.path.join("model/vqvae/",f"num_embedding_{cfg['num_embeddings']}_commitment_cost_{cfg['commitment_cost']}.pt")
        torch.save(model.state_dict(),best_model_path)
    
    print(f"Training Finished. Best model: {best_model_path} (val_loss: {best_val_loss:.6f})")

[001/200 Training] recon=0.311883 vq=0.000941


[001/200 Validation] recon=0.305057 vq=0.000567


[002/200 Training] recon=0.293283 vq=0.000480


[002/200 Validation] recon=0.281284 vq=0.000449


[003/200 Training] recon=0.264101 vq=0.000381


...

[198/200 Validation] recon=0.050939 vq=0.000000


[199/200 Training] recon=0.050779 vq=0.000000


[199/200 Validation] recon=0.050905 vq=0.000000


[200/200 Training] recon=0.050647 vq=0.000000


[200/200 Validation] recon=0.050947 vq=0.000000
Training Finished. Best model: model/vqvae/num_embedding_64_commitment_cost_0.4.pt (val_loss: 0.050905)


In [57]:
model.load_state_dict(torch.load(best_model_path))

test_recon, test_vq, test_loss, index_list = evaluate(model, test_loader, device)
wandb.log({
    f'test_epoch_recon': test_recon,
    f'test_epoch_vq': test_vq
})
print(f"[Test] recon={test_recon:.6f} vq={test_vq:.6f}")

[Test] recon=0.052743 vq=0.000000


In [58]:
wandb.finish()

test_epoch_recon,▁
test_epoch_vq,▁
usage_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_epoch_recon,██▇▇▆▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_epoch_vq,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_epoch_recon,0.05274
test_epoch_vq,0
usage_rate,0.01562
valid_epoch_recon,0.05095
valid_epoch_vq,0


## Get Result

In [60]:
all_x = []
all_assign = []

model.eval()
with torch.no_grad():
  for batch in test_loader:
    X = batch["x"].to(device)
    _, _, indices, _ = model(X)

    all_x.append(X.cpu())
    all_assign.append(indices.cpu())

X = torch.cat(all_x, dim=0)
assign = torch.cat(all_assign, dim=0)
assign

tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]])

In [67]:
## Embedding Extract

In [84]:
wandb.init()

In [87]:
from tqdm import tqdm

# 전체 데이터셋용 DataLoader (shuffle=False 필수)
full_loader = DataLoader(
    ds,
    batch_size=wandb.config.batch_size,
    shuffle=False
)


In [88]:
model.eval()

all_idx = []
all_codes = []
all_embs = []  # z_q 벡터 (옵션)

with torch.no_grad():
    for batch in tqdm(full_loader, desc="Extract VQ codes"):
        X = batch["x"].to(device)          # (B, C, T)
        idxs = batch["idx"]                # 전역 윈도우 index

        x_recon, vq_loss, indices_bt, z_q = model(X)
        # indices_bt: (B, T=1) → 각 윈도우당 하나의 코드
        codes = indices_bt.squeeze(1).cpu().numpy()   # (B,)
        z_vec = z_q.squeeze(-1).cpu().numpy()         # (B, latent_dim)

        all_idx.extend(idxs.cpu().numpy().tolist())
        all_codes.extend(codes.tolist())
        all_embs.extend(z_vec.tolist())


Extract VQ codes: 100%|██████████| 16/16 [00:00<00:00, 470.30it/s]


In [94]:
print(ds[1])

{'x': tensor([[0.5408, 0.5374, 0.2823, 0.5051, 0.6122, 0.9354, 0.8844, 0.7398, 0.8214,
         0.7245, 0.2551, 0.5544, 0.4694],
        [0.7092, 0.7262, 0.5059, 0.6275, 1.0000, 0.9600, 0.9762, 0.8674, 0.9065,
         0.9456, 0.5697, 0.6939, 0.6786],
        [0.3231, 0.0000, 0.1462, 0.3930, 0.6122, 0.8639, 0.7364, 0.6207, 0.6718,
         0.2041, 0.1939, 0.4456, 0.2068],
        [0.5493, 0.2806, 0.5059, 0.6090, 0.9320, 0.8835, 0.7381, 0.8214, 0.7041,
         0.2534, 0.5493, 0.4702, 0.6361],
        [0.4810, 0.5997, 0.1571, 0.1495, 0.4042, 0.0804, 0.0177, 0.1493, 0.0000,
         0.8152, 0.3566, 0.2410, 1.0000]]), 'idx': 1}


In [114]:
import os

chunk_size = ds.chunk_size
rows = []
for idx, code, emb in zip(all_idx, all_codes, all_embs):
    start = idx * chunk_size
    end = start + chunk_size
    ts = normalized_data.iloc[end - 1]["date"]
    ts = ts.date()

    row = {"date": ts, "code": int(code)}
    for j, v in enumerate(emb):
        row[f"z_{j}"] = float(v)
    rows.append(row)

df_vq = pd.DataFrame(rows).sort_values("date").reset_index(drop=True)

# 2) 로컬 임시 파일 경로
local_path = "prepared_data/embedding.csv"
df_vq.to_csv(local_path, index=False, encoding="utf-8-sig")


In [69]:
print(ds[1])

{'x': tensor([[0.5408, 0.5374, 0.2823, 0.5051, 0.6122, 0.9354, 0.8844, 0.7398, 0.8214,
         0.7245, 0.2551, 0.5544, 0.4694],
        [0.7092, 0.7262, 0.5059, 0.6275, 1.0000, 0.9600, 0.9762, 0.8674, 0.9065,
         0.9456, 0.5697, 0.6939, 0.6786],
        [0.3231, 0.0000, 0.1462, 0.3930, 0.6122, 0.8639, 0.7364, 0.6207, 0.6718,
         0.2041, 0.1939, 0.4456, 0.2068],
        [0.5493, 0.2806, 0.5059, 0.6090, 0.9320, 0.8835, 0.7381, 0.8214, 0.7041,
         0.2534, 0.5493, 0.4702, 0.6361],
        [0.4810, 0.5997, 0.1571, 0.1495, 0.4042, 0.0804, 0.0177, 0.1493, 0.0000,
         0.8152, 0.3566, 0.2410, 1.0000]]), 'idx': 1}


# News Concatenation 

In [4]:
import pandas as pd
import json

In [5]:
embedding_path = "prepared_data/embedding.csv"
embedding_df = pd.read_csv(embedding_path)
embedding_df["date"] = pd.to_datetime(embedding_df["date"]).dt.date
embedding_df

,date,code,z_0,z_1,z_2,z_3,z_4,z_5,z_6,z_7
0,2023-01-03,25,0.021594,0.189350,0.005870,0.094773,-9.261945e-28,0.034962,0.401183,0.569945
1,2023-01-04,12,0.102584,0.253331,0.088687,0.262031,1.646256e-24,0.050349,0.120076,0.256553
2,2023-01-05,9,0.034468,0.032504,0.005644,0.347086,-1.079616e-28,0.339522,0.439623,0.407905
3,2023-01-06,7,0.190114,0.087750,0.160866,0.387595,-1.205483e-27,0.435344,0.160252,0.076243
4,2023-01-09,16,0.111109,0.432911,0.087223,0.046769,5.278745e-22,0.038995,0.109769,0.296704
...,...,...,...,...,...,...,...,...,...,...
497,2024-12-24,18,0.247099,0.285105,0.219507,0.215229,1.021299e-29,0.226921,0.026946,0.065848
498,2024-12-26,18,0.247099,0.285105,0.219507,0.215229,1.021299e-29,0.226921,0.026946,0.065848
499,2024-12-27,25,0.021594,0.189350,0.005870,0.094773,-9.261945e-28,0.034962,0.401183,0.569945
500,2024-12-30,7,0.190114,0.087750,0.160866,0.387595,-1.205483e-27,0.435344,0.160252,0.076243


In [6]:
news_path = "processed_news.csv"
news_df = pd.read_csv(news_path)
news_df["date"] = pd.to_datetime(news_df["date"]).dt.date
news_df

,date,text,title,body
0,2021-01-01,Sony to launch PlayStation 5 in India on Febru...,Sony to launch PlayStation 5 in India on Febru...,The Japanese firm said it will begin taking pr...
1,2021-01-01,Sony is launching the PS5 in India on February...,Sony is launching the PS5 in India on February...,PlayStation gamers in India will finally have ...
2,2021-01-01,Huawei removes Tencent games from its Chinese ...,Huawei removes Tencent games from its Chinese ...,Huawei has pulled Tencent games like Arena of ...
3,2021-01-01,Stadia Pro adds 'F1 2020' and 'Hotline Miami'\...,Stadia Pro adds 'F1 2020' and 'Hotline Miami',"To ring in the new year, Google has added four..."
4,2021-01-01,A workout-at-home service report card\n\nHere'...,A workout-at-home service report card,Here's how a number of at-home workout service...
...,...,...,...,...
66148,2023-12-30,Mark Zuckerberg Is Reportedly Building an Unde...,Mark Zuckerberg Is Reportedly Building an Unde...,Meta's Mark Zuckerberg and his wife Priscilla ...
66149,2023-12-30,Ozempic drugmaker to open AI research hub in K...,Ozempic drugmaker to open AI research hub in K...,Novo Nordisk is poised to open a new AI resear...
66150,2023-12-30,"Colombia Should Lower 35% Corporate Tax Rate, ...","Colombia Should Lower 35% Corporate Tax Rate, ...",(Bloomberg) -- Colombia President Gustavo Petr...
66151,2023-12-30,Nvidia Stock Has This Under-the-Radar Artifici...,Nvidia Stock Has This Under-the-Radar Artifici...,Nvidia stock investors are all focused on the ...


In [7]:
group_news_df = news_df.groupby("date")["text"].apply(list).reset_index()
group_news_df

,date,text
0,2021-01-01,[Sony to launch PlayStation 5 in India on Febr...
1,2021-01-02,['Apex Legends' Fight Night event formalizes i...
2,2021-01-03,[Mixtape podcast: Behind the curtain of divers...
3,2021-01-04,"[Known for 5G mmWave testing solutions, Taiwan..."
4,2021-01-05,[Twitter is buying a podcast app to help build...
...,...,...
943,2023-12-20,[Forsaken China Stocks Get a New Look by Top E...
944,2023-12-22,"[Up Over 80%, Is Amazon Stock a Buy for 2024?\..."
945,2023-12-26,[Did Your Holiday Shopping Follow National Tre...
946,2023-12-27,[13 Best IPO Stocks to Buy Heading into 2024\n...


In [8]:
group_news_df["text"] = group_news_df["text"].apply(json.dumps)
group_news_df

,date,text
0,2021-01-01,"[""Sony to launch PlayStation 5 in India on Feb..."
1,2021-01-02,"[""'Apex Legends' Fight Night event formalizes ..."
2,2021-01-03,"[""Mixtape podcast: Behind the curtain of diver..."
3,2021-01-04,"[""Known for 5G mmWave testing solutions, Taiwa..."
4,2021-01-05,"[""Twitter is buying a podcast app to help buil..."
...,...,...
943,2023-12-20,"[""Forsaken China Stocks Get a New Look by Top ..."
944,2023-12-22,"[""Up Over 80%, Is Amazon Stock a Buy for 2024?..."
945,2023-12-26,"[""Did Your Holiday Shopping Follow National Tr..."
946,2023-12-27,"[""13 Best IPO Stocks to Buy Heading into 2024\..."


In [9]:
final_df = embedding_df.merge(group_news_df, on="date", how="inner")

In [10]:
final_df

,date,code,z_0,z_1,z_2,z_3,z_4,z_5,z_6,z_7,text
0,2023-01-03,25,0.021594,0.189350,0.005870,0.094773,-9.261945e-28,0.034962,0.401183,0.569945,"[""INFOLOB Solutions, Inc. Announces Company Re..."
1,2023-01-04,12,0.102584,0.253331,0.088687,0.262031,1.646256e-24,0.050349,0.120076,0.256553,"[""Rover Metals Announces Intent to Re-price Pr..."
2,2023-01-05,9,0.034468,0.032504,0.005644,0.347086,-1.079616e-28,0.339522,0.439623,0.407905,"[""Innovent Announces NMPA's Breakthrough Thera..."
3,2023-01-06,7,0.190114,0.087750,0.160866,0.387595,-1.205483e-27,0.435344,0.160252,0.076243,"[""VRSim Hosts Senator Murphy at East Hartford ..."
4,2023-01-09,16,0.111109,0.432911,0.087223,0.046769,5.278745e-22,0.038995,0.109769,0.296704,"[""Esperion Outlines Upcoming Milestones and An..."
...,...,...,...,...,...,...,...,...,...,...,...
221,2023-12-19,18,0.247099,0.285105,0.219507,0.215229,1.021299e-29,0.226921,0.026946,0.065848,"[""Here's Why I Just Sold These 2 Blue-Chip Sto..."
222,2023-12-20,16,0.111109,0.432911,0.087223,0.046769,5.278745e-22,0.038995,0.109769,0.296704,"[""Forsaken China Stocks Get a New Look by Top ..."
223,2023-12-22,16,0.111109,0.432911,0.087223,0.046769,5.278745e-22,0.038995,0.109769,0.296704,"[""Up Over 80%, Is Amazon Stock a Buy for 2024?..."
224,2023-12-26,9,0.034468,0.032504,0.005644,0.347086,-1.079616e-28,0.339522,0.439623,0.407905,"[""Did Your Holiday Shopping Follow National Tr..."


In [11]:
final_path = "prepared_data/final.csv"
final_df.to_csv(final_path, index=False)

# Silver Labeling

In [3]:
import pandas as pd
from google import genai
from google.genai.errors import APIError
import time

# --- 1. 설정 및 초기화 ---

# ⚠️ 여기에 실제 Gemini API 키를 입력하세요.
# 환경 변수를 사용하는 것을 권장하지만, 테스트를 위해 직접 입력할 수도 있습니다.
API_KEY = ""
try:
    # client 객체는 for 루프 밖에서 한 번만 초기화됩니다.
    client = genai.Client(api_key=API_KEY)
    model = 'gemini-2.5-flash'  # 빠른 응답을 위해 flash 모델 사용
except Exception as e:
    print(f"Gemini 클라이언트 초기화 오류: {e}")
    exit()

# --- 2. 데이터 로드 ---

file_path = 'prepared_data/final.csv'
try:
    df = pd.read_csv(file_path)
    print(f"✅ 파일 '{file_path}' 로드 완료. 총 {len(df)}개 행.")
except FileNotFoundError:
    print(f"❌ 오류: 파일 '{file_path}'를 찾을 수 없습니다. 경로를 확인해주세요.")
    exit()
except Exception as e:
    print(f"❌ 오류: CSV 파일을 읽는 중 문제가 발생했습니다: {e}")
    exit()

# 그룹화할 키 컬럼
GROUPING_KEY = 'code'

if GROUPING_KEY not in df.columns:
    print(f"❌ 오류: 데이터프레임에 그룹화 키인 '{GROUPING_KEY}' 열이 없습니다.")
    exit()

# --- 3. Silver Label 생성을 위한 함수 정의 ---

def generate_silver_label(text_list, code_value, client_obj, model_name, prompt_input):
    """
    주어진 텍스트 리스트를 Gemini API를 사용하여 요약합니다.
    (API 클라이언트 객체를 인수로 받도록 수정)
    """
    if not text_list:
        return ""

    # 텍스트들을 하나의 문자열로 결합
    combined_text = "\n---\n".join(text_list)
    combined_text = combined_text[:100000]
    # 프롬프트 정의
    prompt = f"""
    아래에 제공된 텍스트들은 공통적으로 코드 값 {code_value}를 가지는 뉴스 기사/정보의 일부입니다.
    
    이 모든 텍스트의 핵심 내용과 주제를 파악하여, 100~200자 사이의 하나의 **한국어 요약(Silver Label)**으로 정리해 주세요.
    요약은 다음 형식을 따릅니다: "핵심 주제: (핵심 내용)"
    
    --- 텍스트 목록 ---
    {combined_text}
    """

    try:
        # Gemini API 호출
        response = client_obj.models.generate_content(
            model=model_name,
            contents=prompt
        )
        return response.text.strip()
    
    # 토큰 만료 또는 기타 API 오류 발생 시
    except APIError as e:
        # 오류 상세 정보를 반환하여 나중에 원인을 분석할 수 있게 함
        return f"요약 오류: API Error - {e}"
    except Exception as e:
        return f"요약 오류: Unexpected Error - {e}"

# --- 4. 그룹별 요약 및 Silver Label 생성 (지연/반복문 적용) ---

# GROUPING_KEY를 기준으로 그룹화
grouped = df.groupby(GROUPING_KEY)['text'].apply(list).reset_index(name='text_list')
print(f"⚙️ {GROUPING_KEY} 값 {len(grouped)}개로 그룹화 완료.")

# 'silver_label' 열을 미리 초기화합니다.
grouped['silver_label'] = None 

print("✨ 그룹별 Silver Label 생성 중... (API 호출 사이에 **1분 지연**이 적용됩니다.)")

# 인덱스를 리스트로 변환하여 마지막 인덱스를 쉽게 확인
group_indices = grouped.index.tolist()
total_groups = len(group_indices)


prompts = [
  {
    "prompt_number": 1,
    "prompt_context": "상세 분석 및 항목 강제: 요약에 반드시 '주요 산업 분야/기술 트렌드', '주요 이슈/활동 유형', '전반적인 시장 분위기/영향' 세 가지 요소를 포함하도록 강제하여 구체적인 분석 데이터를 확보합니다.",
    "prompt_text": "아래 제공된 텍스트들은 공통적으로 코드 값 {code_value}를 가지는 뉴스 기사/정보의 일부입니다.\n\n당신은 전문 뉴스 분석가입니다. 이 모든 텍스트를 분석하여, 다음 세 가지 핵심 항목을 반드시 포함하는 100~200자 사이의 하나의 **한국어 요약(Silver Label)**으로 정리해 주세요.\n\n1. **언급된 주요 산업 분야 또는 기술 트렌드** (예: IT, 헬스케어, ESG, AI 등)\n2. **주요 이슈/활동의 유형** (예: 인수합병, 신제품 출시, 실적 발표, 정책 변화 등)\n3. **전반적인 시장 분위기 또는 영향** (예: 성장 기대, 불확실성 증대, 규제 강화 등)\n\n요약은 반드시 다음 형식을 따르며, 구체적이어야 합니다:\n\"핵심 주제: (핵심 내용. 위 3가지 항목이 모두 녹아 있어야 함)\"\n\n--- 텍스트 목록 ---\n{combined_text}",
    "output_name": "silver_label_detailed_analysis.csv"
  },
  {
    "prompt_number": 2,
    "prompt_context": "시장 동향 중심 요약: 개별 기업 활동보다는 거시적 관점의 '시장 변화'와 '위험 요소'에 초점을 맞춰 요약하도록 유도하여 트렌드 레이블을 추출합니다.",
    "prompt_text": "아래 제공된 텍스트들은 공통적으로 코드 값 {code_value}를 가지는 뉴스 기사/정보의 일부입니다.\n\n이 모든 텍스트를 기반으로, 해당 코드와 관련된 **가장 중요한 거시적 시장 동향**과 **잠재적 위험/기회 요소**를 100~200자 사이의 하나의 **한국어 요약(Silver Label)**으로 정리해 주세요.\n\n**[핵심 요구사항]**\n1. **경제 환경/정책 변화**가 포함되어야 합니다.\n2. **개별 기업명 언급은 최소화**하고 산업 전반의 방향성을 제시해야 합니다.\n\n요약 형식: \"핵심 주제: (시장 변화와 주요 리스크/기회에 초점 맞춘 내용)\"\n\n--- 텍스트 목록 ---\n{combined_text}",
    "output_name": "silver_label_market_trend.csv"
  },
  {
    "prompt_number": 3,
    "prompt_context": "감성/태도 평가: 텍스트의 전반적인 분위기(긍정/부정/중립)를 파악하고, 그 근거가 되는 활동(투자 vs. 하락)을 명시하도록 요구하여 감성 데이터 레이블을 추출합니다.",
    "prompt_text": "아래 제공된 텍스트들을 분석하여, 해당 코드 {code_value}와 관련된 활동의 **전반적인 감성(긍정/부정/중립)**을 파악하고 그 근거를 요약하세요.\n\n요약은 다음 네 가지 요소를 모두 포함하는 100~200자 사이의 하나의 **한국어 요약(Silver Label)**으로 정리해 주세요.\n\n1. **가장 빈번하게 언급된 기업 활동** (예: 성장, 투자, 하락, 구조조정)\n2. **전반적인 감성 평가** (긍정, 부정, 또는 혼재)\n3. **감성 평가의 근거**가 되는 주요 사건 (긍정적/부정적)\n\n요약 형식: \"핵심 주제: (전반적인 감성 평가와 그 근거를 명확히 제시한 내용)\"\n\n--- 텍스트 목록 ---\n{combined_text}",
    "output_name": "silver_label_sentiment_focus.csv"
  }
]

for index, p in enumerate(prompts):
    concated_prompt = p["prompt_context"] + p["prompt_text"]

# 반복문을 사용하여 한 줄씩 처리
    for i, index in enumerate(group_indices):
        row = grouped.loc[index]
        text_list = row['text_list']
        group_key = row[GROUPING_KEY]
        
        print(f"\n🚀 {i+1}/{total_groups} 처리 중 - Code: {group_key} (텍스트 {len(text_list)}개)")

        # 1. generate_silver_label 함수 호출
        silver_label = generate_silver_label(text_list, group_key, client, model, concated_prompt)
        
        # 2. 결과 저장
        grouped.loc[index, 'silver_label'] = silver_label
    
        if silver_label.startswith("요약 오류:"):
            print(f"❌ 오류 발생: {silver_label}")
        else:
            print("✅ Silver Label 생성 및 저장 완료.")

        time.sleep(0.5)
        
    print("\n✅ Silver Label 생성 완료.")

    # --- 5. 원본 데이터프레임에 Silver Label 병합 ---

    # 병합을 위해 'silver_label'과 GROUPING_KEY만 포함하는 데이터프레임 준비
    silver_labels_df = grouped[[GROUPING_KEY, 'silver_label']]

    # 원본 데이터프레임에 병합
    df_merged = pd.merge(df.drop(columns=['text']), silver_labels_df, on=GROUPING_KEY, how='left')

    # 'silver_label'을 새 'text'로 사용하고 이름을 변경
    df_merged.rename(columns={'silver_label': 'text'}, inplace=True)

    # 열 순서 조정 (원본과 유사하게)
    cols = list(df_merged.columns)
    cols.remove('text')
    df_final = df_merged[cols + ['text']]

    # --- 6. 결과 저장 ---

    output_file_path = p["output_name"]
    try:
        df_final.to_csv(output_file_path, index=False, encoding='utf-8')
        print(f"\n🎉 최종 결과가 '{output_file_path}'에 저장되었습니다.")
        print("👉 이제 이 파일을 사용하여 후속 분석을 진행할 수 있습니다.")
    except Exception as e:
        print(f"❌ 오류: 최종 CSV 파일 저장 중 문제가 발생했습니다: {e}")

✅ 파일 'prepared_data/final.csv' 로드 완료. 총 226개 행.
⚙️ code 값 7개로 그룹화 완료.
✨ 그룹별 Silver Label 생성 중... (API 호출 사이에 **1분 지연**이 적용됩니다.)

🚀 1/7 처리 중 - Code: 7 (텍스트 41개)
✅ Silver Label 생성 및 저장 완료.

🚀 2/7 처리 중 - Code: 9 (텍스트 47개)
✅ Silver Label 생성 및 저장 완료.

🚀 3/7 처리 중 - Code: 12 (텍스트 18개)
✅ Silver Label 생성 및 저장 완료.

🚀 4/7 처리 중 - Code: 16 (텍스트 24개)
✅ Silver Label 생성 및 저장 완료.

🚀 5/7 처리 중 - Code: 18 (텍스트 35개)
✅ Silver Label 생성 및 저장 완료.

🚀 6/7 처리 중 - Code: 25 (텍스트 51개)
✅ Silver Label 생성 및 저장 완료.

🚀 7/7 처리 중 - Code: 26 (텍스트 10개)
✅ Silver Label 생성 및 저장 완료.

✅ Silver Label 생성 완료.

🎉 최종 결과가 'silver_label_detailed_analysis.csv'에 저장되었습니다.
👉 이제 이 파일을 사용하여 후속 분석을 진행할 수 있습니다.

🚀 1/7 처리 중 - Code: 7 (텍스트 41개)
✅ Silver Label 생성 및 저장 완료.

🚀 2/7 처리 중 - Code: 9 (텍스트 47개)
✅ Silver Label 생성 및 저장 완료.

🚀 3/7 처리 중 - Code: 12 (텍스트 18개)
✅ Silver Label 생성 및 저장 완료.

🚀 4/7 처리 중 - Code: 16 (텍스트 24개)
✅ Silver Label 생성 및 저장 완료.

🚀 5/7 처리 중 - Code: 18 (텍스트 35개)
✅ Silver Label 생성 및 저장 완료.

🚀 6/7 처리 중 - Code: 25 (텍스트 51개)
✅ Silver

KeyboardInterrupt: 

# Prompting

In [6]:
import os
import math
from typing import List

import numpy as np
import pandas as pd

print("pandas version:", pd.__version__)

# s3 기본 설정
# BUCKET = "sagemaker-us-west-2-327784329358"
# S3_BASE = f"s3://{BUCKET}"

# 입력 데이터: S3에서 바로 읽기
# PRICE_OHLCV_PATH = f"{S3_BASE}/prepared_data/price_ohlcv.csv"
# VQ_CODE_CSV_PATH = f"{S3_BASE}/vq_vae_outputs/vq_codes_spy.csv"
# NEWS_CSV_PATH = f"{S3_BASE}/prepared_data/sp500_headlines_2008_2024.csv"

# 출력 JSONL: 로컬에 만들고 S3로 업로드
# OUTPUT_JSONL_LOCAL_PATH = "/tmp/market_commentary_train.jsonl"
# OUTPUT_JSONL_S3_KEY = "prepared_data/market_commentary_train.jsonl"

# Data Path
OUTPUT_JSONL_LOCAL_PATH = "prepared_data/train.jsonl"
SILVER_LABEL_PATH = "silver_label_market_trend.csv"
PRICE_OHLCV_PATH = "spy_2023_2024.csv"

pandas version: 2.3.3


In [22]:
df_silver_label = pd.read_csv(SILVER_LABEL_PATH)
df_price = pd.read_csv(PRICE_OHLCV_PATH)

In [23]:
# required_cols = {"date", "close"}
# if not required_cols.issubset(df_price.columns):
    # raise ValueError("prices_ohlcv.csv에 date, close 컬럼이 필요합니다.")

df_silver_label['date'] = pd.to_datetime(df_silver_label['date']).dt.date

df_price['date'] = pd.to_datetime(df_price['date'])
df_price['day'] = df_price['date'].dt.date
df_price = df_price.groupby('day').tail(1)

def make_summary(row):
    return (
        f"open: {row['1. open']}, "
        f"high: {row['2. high']}, "
        f"low: {row['3. low']}, "
        f"close: {row['4. close']}, "
        f"volume: {row['5. volume']}"
    )

df_price["ohlcv_summary"] = df_price.apply(make_summary, axis=1)
df_price = df_price[['day', 'ohlcv_summary']].copy()
df_price = df_price.rename(columns={'day': 'date'})
df_price['date'] = pd.to_datetime(df_price['date']).dt.date

# df_price["date"] = pd.to_datetime(df_price["date"])
# df_price = df_price.sort_values("date").reset_index(drop=True)

# 여기서 수익률, 변동성 등 파생 피처 생성
# df_price["ret_1d"] = df_price["close"].pct_change()
# df_price["log_ret"] = np.log(df_price["close"]).diff()
# df_price["close_ma_5"] = df_price["close"].rolling(window=5).mean()
# df_price["close_ma_20"] = df_price["close"].rolling(window=20).mean()
# df_price["vol_5"] = df_price["log_ret"].rolling(window=5).std()
# df_price["vol_20"] = df_price["log_ret"].rolling(window=20).std()

# NaN 제거
# df_price = df_price.dropna().reset_index(drop=True)
# print("[PRICE] head:\n", df_price.head())


# VQ-VAE 코드와 price join
# df_vq = pd.read_csv(VQ_CODE_CSV_PATH)
# df_vq["date"] = pd.to_datetime(df_vq["date"])

# df_merged = pd.merge(df_vq, df_price, on="date", how="inner")
# df_merged = df_merged.sort_values("date").reset_index(drop=True)

# News Headline join
# df_news = pd.read_csv("NEWS_CSV_PATH")
# df_news["Date"] = pd.to_datetime(df_news["Date"])

""" news_grouped = (
    df_news.groupby("Date")["Title"]
    .apply(list)
    .reset_index()
    .rename(columns={"Date": "date", "Title": "titles"})
) """

# merge all
df_all = pd.merge(
    df_silver_label,
    df_price,                # VQ 코드 + 가격 요약
    on="date",
    how="left",
)

In [31]:
def code_to_str(row):
    z_values = [row[f"z_{i}"] for i in range(8)]
    z_str = ", ".join([f"{v:.6f}" for v in z_values])
    return f"{z_str}"

df_all["code_str"] = df_all.apply(code_to_str, axis=1)

In [32]:
df_all.head()

,date,code,z_0,z_1,z_2,z_3,z_4,z_5,z_6,z_7,text,ohlcv_summary,prompt,completion,code_str
0,2023-01-03,25,0.021594,0.189350,0.005870,0.094773,-9.261945e-28,0.034962,0.401183,0.569945,"핵심 주제: 기업 경영, 시장 동향 및 신기술\n다양한 기업들이 경영진 개편, M&...","open: 366.6838, high: 367.5803, low: 366.6066,...","### 코드\n25\n\n### OHLCV 시계열\nopen: 366.6838, h...","핵심 주제: 기업 경영, 시장 동향 및 신기술\n다양한 기업들이 경영진 개편, M&...","0.021594, 0.189350, 0.005870, 0.094773, -0.000..."
1,2023-01-04,12,0.102584,0.253331,0.088687,0.262031,1.646256e-24,0.050349,0.120076,0.256553,핵심 주제: 2023년 초 글로벌 비즈니스 및 기술 동향\n\n2023년 초 다양한...,"open: 368.9878, high: 370.1735, low: 367.4996,...","### 코드\n12\n\n### OHLCV 시계열\nopen: 368.9878, h...",핵심 주제: 2023년 초 글로벌 비즈니스 및 기술 동향\n\n2023년 초 다양한...,"0.102584, 0.253331, 0.088687, 0.262031, 0.0000..."
2,2023-01-05,9,0.034468,0.032504,0.005644,0.347086,-1.079616e-28,0.339522,0.439623,0.407905,핵심 주제: 2023년 초 다양한 산업의 혁신 및 시장 동향: CES 2023을 통...,"open: 365.9704, high: 366.2837, low: 365.2811,...","### 코드\n9\n\n### OHLCV 시계열\nopen: 365.9704, hi...",핵심 주제: 2023년 초 다양한 산업의 혁신 및 시장 동향: CES 2023을 통...,"0.034468, 0.032504, 0.005644, 0.347086, -0.000..."
3,2023-01-06,7,0.190114,0.087750,0.160866,0.387595,-1.205483e-27,0.435344,0.160252,0.076243,"핵심 주제: 광범위한 산업의 시장 성장 전망, 기업의 재무 성과, 투자 및 자금 조...","open: 374.4007, high: 375.0707, low: 373.7789,...","### 코드\n7\n\n### OHLCV 시계열\nopen: 374.4007, hi...","핵심 주제: 광범위한 산업의 시장 성장 전망, 기업의 재무 성과, 투자 및 자금 조...","0.190114, 0.087750, 0.160866, 0.387595, -0.000..."
4,2023-01-09,16,0.111109,0.432911,0.087223,0.046769,5.278745e-22,0.038995,0.109769,0.296704,핵심 주제: 글로벌 기업 경영 활동 및 시장 전망. 다양한 산업 분야에서 기업들의 ...,"open: 374.3863, high: 374.5694, low: 373.7211,...","### 코드\n16\n\n### OHLCV 시계열\nopen: 374.3863, h...",핵심 주제: 글로벌 기업 경영 활동 및 시장 전망. 다양한 산업 분야에서 기업들의 ...,"0.111109, 0.432911, 0.087223, 0.046769, 0.0000..."


In [39]:
def build_prompt(code_value: int, code_vector: str, ohlcv_summary: str):
    prompt = (
        "### 코드북\n" + str(code_value) +"\n\n"
        "### 코드북 벡터\n" + str(code_vector) + "\n\n"
        "### OHLCV 시계열\n" + ohlcv_summary + "\n\n"

        "위의 Code 벡터 정보와 OHLCV 시계열을 바탕으로, 해당 구간의 시장 움직임을 설명할 수 있는 한국어 뉴스 요약을 2~3 문장으로 작성하라."
        "데이터에 기반한 보수적 설명만 사용하고, 입력에 없는 사건을 새로 만들어내지 마라."
    )
    return prompt
    
# 프롬프트 컬럼 생성 (레이블은 placeholder)
df_all["prompt"] = df_all.apply(
    lambda row: build_prompt(
        code_value=row["code"],
        code_vector=row["code_str"],
        ohlcv_summary=row["ohlcv_summary"],
    ),
    axis=1,
)

# 실제 프로젝트에서는 아래 completion을 사람/teacher LLM으로 채워넣어야 함
# silver label을 llm api로 사용하거나 사람 손으로 gole label을 붙여 넣어야 함
df_all["completion"] = df_all["text"]

print("[ALL with prompt/completion] head:\n",
      df_all[["date", "code", "prompt", "completion"]].head())


[ALL with prompt/completion] head:
          date  code                                             prompt  \
0  2023-01-03    25  ### 코드북\n25\n\n### 코드북 벡터\n0.021594, 0.189350,...   
1  2023-01-04    12  ### 코드북\n12\n\n### 코드북 벡터\n0.102584, 0.253331,...   
2  2023-01-05     9  ### 코드북\n9\n\n### 코드북 벡터\n0.034468, 0.032504, ...   
3  2023-01-06     7  ### 코드북\n7\n\n### 코드북 벡터\n0.190114, 0.087750, ...   
4  2023-01-09    16  ### 코드북\n16\n\n### 코드북 벡터\n0.111109, 0.432911,...   

                                          completion  
0  핵심 주제: 기업 경영, 시장 동향 및 신기술\n다양한 기업들이 경영진 개편, M&...  
1  핵심 주제: 2023년 초 글로벌 비즈니스 및 기술 동향\n\n2023년 초 다양한...  
2  핵심 주제: 2023년 초 다양한 산업의 혁신 및 시장 동향: CES 2023을 통...  
3  핵심 주제: 광범위한 산업의 시장 성장 전망, 기업의 재무 성과, 투자 및 자금 조...  
4  핵심 주제: 글로벌 기업 경영 활동 및 시장 전망. 다양한 산업 분야에서 기업들의 ...  


In [40]:
# import boto3
import json

def export_jsonl(df: pd.DataFrame, out_path: str) -> None:
    os.makedirs(os.path.dirname(out_path) or ".", exist_ok=True)
    with open(out_path, "w", encoding="utf-8") as f:
        for row in df.itertuples():
            rec = {
                "prompt": getattr(row, "prompt"),
                "completion": getattr(row, "completion"),
                "date": str(getattr(row, "date")),
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    print(f"Saved JSONL to {out_path}")

export_jsonl(df_all, OUTPUT_JSONL_LOCAL_PATH)

# upload to s3 bucket
# s3 = boto3.client("s3")
# s3.upload_file(OUTPUT_JSONL_LOCAL_PATH, BUCKET, OUTPUT_JSONL_S3_KEY)

# print(f"Uploaded to s3://{BUCKET}/{OUTPUT_JSONL_S3_KEY}")

Saved JSONL to prepared_data/train.jsonl


# LLM Fine Tuning

In [1]:
import os
import json
from dataclasses import dataclass
from typing import Dict, List

import random
import numpy as np
import torch
from torch.utils.data import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    default_data_collator,
)

from peft import LoraConfig, get_peft_model


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
@dataclass
class TrainConfig:
    # 1) 모델 & 데이터
    model_name_or_path: str = "meta-llama/Meta-Llama-3-8B-Instruct"  # 원하는 베이스 LLM 이름
    train_jsonl: str = "prepared_data/train.jsonl"          # 학습시킬 json prompt
    
    # 결과 저장 위치: EBS 볼륨 내 현재 작업 디렉토리 기준 "./model"
    output_dir: str = "./model"

    # 2) 토크나이즈 & 길이
    max_length: int = 200

    # 3) 학습 하이퍼파라미터
    per_device_train_batch_size: int = 1
    gradient_accumulation_steps: int = 16
    num_train_epochs: int = 3
    learning_rate: float = 2e-4
    warmup_ratio: float = 0.03

    # 4) 로깅 & 저장
    logging_steps: int = 50
    save_steps: int = 500
    seed: int = 42
    # 5) 기타
    dataloader_num_workers: int = 2  # Colab이면 0~2 정도로 유지

    # device_map="auto"
    # load_in_4bit=True

cfg = TrainConfig()
cfg


TrainConfig(model_name_or_path='meta-llama/Meta-Llama-3-8B-Instruct', train_jsonl='prepared_data/train.jsonl', output_dir='./model', max_length=200, per_device_train_batch_size=1, gradient_accumulation_steps=16, num_train_epochs=3, learning_rate=0.0002, warmup_ratio=0.03, logging_steps=50, save_steps=500, seed=42, dataloader_num_workers=2)

In [3]:
def resolve_jsonl_path(path: str) -> str:
    """
    - s3://bucket/key 형식이면 /tmp/train.jsonl 로 다운로드 후 해당 경로 반환
    - 그 외에는 로컬 경로 그대로 반환
    """
    if path.startswith("s3://"):
        no_scheme = path[5:]
        bucket, key = no_scheme.split("/", 1)

        local_path = "/tmp/train.jsonl"
        os.makedirs(os.path.dirname(local_path), exist_ok=True)

        s3 = boto3.client("s3")
        s3.download_file(bucket, key, local_path)
        print(f"[INFO] downloaded {path} -> {local_path}")
        return local_path
    else:
        return path


def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


seed_everything(cfg.seed)


In [4]:
class JsonlSftDataset(Dataset):
    """
    SFT용 JSONL Dataset.
    각 row는 {"prompt": "...", "completion": "..."} 형태여야 함.
    반환되는 input_ids, attention_mask, labels는 모두 동일 길이를 보장해야 한다.
    """

    def __init__(self, jsonl_path, tokenizer, max_length=1024):
        self.samples = []
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                if not line.strip():
                    continue
                obj = json.loads(line)
                self.samples.append(obj)

        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        item = self.samples[idx]
        prompt = " ".join(str(item["prompt"]).split())
        completion = " ".join(str(item["completion"]).split())

        # 1) 프롬프트 + completion 합치기
        full_text = prompt + completion

        # 2) tokenizer로 일괄 encoding (truncation=True)
        enc = self.tokenizer(
            full_text,
            truncation=True,
            max_length=self.max_length,
            add_special_tokens=True,
        )

        input_ids = enc["input_ids"]
        attn_mask = enc["attention_mask"]

        # 3) prompt만 tokenize해서 길이 파악
        prompt_ids = self.tokenizer(
            prompt,
            add_special_tokens=False,
        )["input_ids"]

        # BOS 토큰 보정
        bos_extra = 1 if (len(input_ids) > 0 and input_ids[0] == self.tokenizer.bos_token_id) else 0

        prompt_len = len(prompt_ids) + bos_extra

        # 4) labels = input_ids 복사
        labels = input_ids.copy()

        # prompt 구간 -100
        for i in range(prompt_len):
            if i < len(labels):
                labels[i] = -100

        # ※ 여기서 중요한 부분: 
        # input_ids, attention_mask, labels 모두 정확히 동일 길이
        # collator가 나중에 batch padding 할 것이므로 Dataset에서는 길이를 맞추기만 하면 된다.

        assert len(input_ids) == len(labels), f"label mismatch: {len(input_ids)} vs {len(labels)}"

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attn_mask, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long),
        }


In [5]:
from huggingface_hub import login
from huggingface_hub import HfApi
from huggingface_hub import hf_hub_download

api = HfApi()

try:
    hf_hub_download(
        repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
        filename="config.json"
    )
    print("Access OK!")
except Exception as e:
    print("Access error:", e)

import requests

HF_TOKEN = "hf_xxxxxxxxx"   # 너의 것 넣기
headers = {"Authorization": f"Bearer {HF_TOKEN}"}

url = "https://huggingface.co/api/models/meta-llama/Meta-Llama-3-8B-Instruct"
res = requests.get(url, headers=headers)

print(res.status_code)
# print(res.json())


Access OK!
200


In [5]:
# 1) 토크나이저
tokenizer = AutoTokenizer.from_pretrained(
    cfg.model_name_or_path,
    use_fast=True,
)

# pad 토큰 세팅 (없는 모델들 대비)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [6]:
# 2) 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name_or_path,
    torch_dtype=torch.bfloat16,
    device_map="cuda",  # GPU 있으면 GPU, 없으면 CPU
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False  # 학습 시에는 False 권장

# 3) LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # LLaMA 계열 기준. 다른 모델이면 target_modules 이름만 바꿔주면 됨.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

trainable params: 6,815,744 || all params: 8,037,076,992 || trainable%: 0.0848


In [8]:
import torch
from torch.nn.utils.rnn import pad_sequence

# pad token이 없는 모델이면 eos를 pad로 재사용
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

def sft_collate_fn(batch):
    """
    batch: list of {"input_ids": 1D tensor, "attention_mask": 1D tensor, "labels": 1D tensor}
    를 받아서, padding된 배치를 반환.
    """

    input_ids_list = [item["input_ids"] for item in batch]
    attention_mask_list = [item["attention_mask"] for item in batch]
    labels_list = [item["labels"] for item in batch]

    # 배치 안에서 가장 긴 길이에 맞춰 패딩
    input_ids_padded = pad_sequence(
        input_ids_list,
        batch_first=True,
        padding_value=tokenizer.pad_token_id,
    )

    attention_mask_padded = pad_sequence(
        attention_mask_list,
        batch_first=True,
        padding_value=0,          # 패딩 위치는 0
    )

    labels_padded = pad_sequence(
        labels_list,
        batch_first=True,
        padding_value=-100,       # loss에서 무시될 값
    )

    return {
        "input_ids": input_ids_padded,
        "attention_mask": attention_mask_padded,
        "labels": labels_padded,
    }


In [9]:
# S3 또는 로컬 경로를 실제 로컬 파일로 resolve
train_jsonl_local = resolve_jsonl_path(cfg.train_jsonl)
print("Train JSONL local path:", train_jsonl_local)

# Dataset 생성
from torch.utils.data import Subset
from transformers import DataCollatorWithPadding

# 1) 전체 dataset 생성
full_dataset = JsonlSftDataset(
    jsonl_path=train_jsonl_local,
    tokenizer=tokenizer,
    max_length=cfg.max_length,
)

# 2) train / val split (예: 90% / 10%)
n = len(full_dataset)
val_ratio = 0.1
val_size = max(1, int(n * val_ratio))

indices = list(range(n))
import random
random.shuffle(indices)

val_indices = indices[:val_size]
train_indices = indices[val_size:]

train_dataset = Subset(full_dataset, train_indices)
eval_dataset = Subset(full_dataset, val_indices)

print(f"Total samples: {n}, train: {len(train_dataset)}, val: {len(eval_dataset)}")

# 3) collator (우리가 만든 labels를 그대로 유지)
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=True,           # 배치 내 최장 길이에 맞춰 패딩
    # max_length=cfg.max_length,  # (선택) 강제로 이 길이까지 패딩/컷팅하고 싶으면 설정
)


Train JSONL local path: prepared_data/train.jsonl
Total samples: 226, train: 204, val: 22


In [10]:
import transformers
print(transformers.__version__)


4.57.3


In [12]:
# trainer
from transformers import TrainingArguments

os.makedirs(cfg.output_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=cfg.output_dir,
    overwrite_output_dir=True,

    num_train_epochs=cfg.num_train_epochs,
    per_device_train_batch_size=cfg.per_device_train_batch_size,
    gradient_accumulation_steps=cfg.gradient_accumulation_steps,
    learning_rate=cfg.learning_rate,
    warmup_ratio=cfg.warmup_ratio,

    logging_steps=cfg.logging_steps,
    save_steps=cfg.save_steps,
    save_total_limit=3,

    fp16=False,
    bf16=True,

    seed=cfg.seed,
    report_to="none",
    dataloader_num_workers=cfg.dataloader_num_workers,

    # 추가: epoch마다 eval_loss 계산
    eval_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,   # ← 추가
    data_collator=data_collator,
)

print("Trainer ready.")



Trainer ready.


In [ ]:
for i in range(0, len(train_dataset)):
    
    sample = train_dataset[i]
    print(len(sample["input_ids"]), len(sample["labels"]))

In [14]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss
1,No log,0.699912
2,No log,0.151138
3,No log,0.119790


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=39, training_loss=0.8509030464367989, metrics={'train_runtime': 139.1183, 'train_samples_per_second': 4.399, 'train_steps_per_second': 0.28, 'total_flos': 5516622161510400.0, 'train_loss': 0.8509030464367989, 'epoch': 3.0})

In [15]:
import os
from datetime import datetime

def save_metrics_with_timestamp(df, prefix, out_dir="./logs"):
    # 1) 디렉토리 생성
    os.makedirs(out_dir, exist_ok=True)

    # 2) 타임스탬프 생성
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")

    # 3) 파일 경로 생성
    filename = f"{prefix}_{ts}.csv"
    path = os.path.join(out_dir, filename)

    # 4) CSV 저장
    df.to_csv(path, index=False, encoding="utf-8-sig")

    print(f"Saved metrics to: {path}")
    return path

In [16]:
#epcoh별 eval_loss와 perplexity 정리
import math
import pandas as pd

log_history = trainer.state.log_history

rows = []
for log in log_history:
    # eval 단계 로그만 골라서 사용
    if "eval_loss" in log:
        epoch = log.get("epoch", None)
        eval_loss = log["eval_loss"]
        perplexity = math.exp(eval_loss)
        rows.append({
            "epoch": epoch,
            "eval_loss": eval_loss,
            "perplexity": perplexity,
        })

df_metrics = pd.DataFrame(rows)
filename = "lora_metrics"
save_metrics_with_timestamp(df_metrics, filename)
print(df_metrics)


Saved metrics to: ./logs/lora_metrics_20251206_031304.csv
   epoch  eval_loss  perplexity
0    1.0   0.699912    2.013576
1    2.0   0.151138    1.163157
2    3.0   0.119790    1.127260


In [17]:
trainer.save_model(cfg.output_dir)      # LoRA 어댑터 포함한 모델 저장
tokenizer.save_pretrained(cfg.output_dir)

print("LoRA fine-tuning 완료. 저장 위치:", cfg.output_dir)


LoRA fine-tuning 완료. 저장 위치: ./model


In [6]:
#simple test
from peft import PeftModel

# 베이스 모델 다시 로드 (새 세션이거나, 검증용)
base_model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name_or_path,
    torch_dtype=torch.float16,
    device_map="cuda",
)
base_model.config.pad_token_id = tokenizer.pad_token_id
base_model.config.use_cache = True  # inference에서는 True로 켜도 됩니다.

lora_model = PeftModel.from_pretrained(
    base_model,
    cfg.output_dir,
)
lora_model.eval()

def generate_comment(prompt_text: str, max_new_tokens: int = 256):
    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=cfg.max_length,
    ).to(lora_model.device)

    with torch.no_grad():
        outputs = lora_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


test_prompt = """당신은 미국 주식 애널리스트다.

[종목 코드] NVDA
[기간] 2025-12-05

[OHLCV 요약]
- 시가: 370.5398
- 고가: 372.5257
- 저가: 367.9274
- 종가: 367.9755
- 거래량: 7391068.0

[VQ 코드 시퀀스]
[0.021593764424324,	0.1893500536680221,	0.0058695869520306,	0.0947726070880889,	-9.261945399163935e-28,	0.0349617674946785,	0.4011827707290649,	0.5699447393417358]

[뉴스 헤드라인]
- "나스닥 종합지수는 화요일 1.5% 이상 폭락한 후 수요일 1.2% 하락했습니다. S&P 500과 다우존스 산업평균지수는 모두 0.8% 하락했습니다."

위 정보를 종합하여 300~400자 분량의 시황 코멘트를 작성하라.

### 답변:
"""
comment = generate_comment(test_prompt)
print(comment)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

당신은 미국 주식 애널리스트다.

[종목 코드] NVDA
[기간] 2025-12-05

[OHLCV 요약]
- 시가: 370.5398
- 고가: 372.5257
- 저가: 367.9274
- 종가: 367.9755
- 거래량: 7391068.0

[VQ 코드 시퀀스]
[0.021593764424324,	0.1893500536680221,	0.0058695869520306,	0.0947726070880889,	-9.261945399163935e-28,	0.0349617674946785,	0.4011827707290649,	0.5699447393417358]

[뉴스 헤드라인]
- "나스닥 종합지수는 화요일 1.3% 상승, 기술株 주도"
- "오피스 소프트웨어 기업, 다양한 제품 출시 계획 발표"
- "신기술 기업, 다양한 인수합병 및 투자 유치 발표"

핵심 주제: 2025년 초 다양한 산업의 혁신 및 성장 전망. 기업들은 다양한 제품 및 서비스 출시, 인수합병 및 투자 유치 등을 통해 기업 가치 및 성장 전망을 높이고 있습니다.

핵심 주제 주제: 기업의 혁신 및 성장 전망, 시장 동향 및 기업 실적. 다양한 산업의 기업들이 다양한 제품 및 서비스 출시, 인수합병 및 투자 유치 등을 통해 기업 가치 및 성장 전망을 높이고 있습니다. 기업들의 혁신 및 성장 전망은 시장 동향 및 기업 실적과 직접적으로 연관이 있습니다.

핵심 주제 주제: 기업의 혁신 및 성장 전망, 시장 동향 및 기업 실적. 다양한 산업의 기업들이 다양한 제품 및 서비스 출시, 인수합병 및 투자 유치 등을 통해 기업 가치 및 성장 전망을 높이고 있습니다. 기업들의 혁


# Evaluation